In [44]:
# 1. Импорт библиотек

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
# 2. Выгружаем данные

df_train = pd.read_csv('train.csv', index_col='ID')
df_test = pd.read_csv('test.csv', index_col='ID')

In [46]:
df_train

,url,title,label
ID,,,
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0
1,www.kp.by,Эта песня стала известна многим телезрителям б...,0
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,0
3,colorbox.spb.ru,Не Беси Меня Картинки,0
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,0
...,...,...,...
135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,0
135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....,0
135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati...",0


In [47]:
df_test

,url,title
ID,,
135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
135313,2gis.ru,67
...,...,...
300682,etp.armtek.ru,Armtek - запчасти для грузовых и легковых авто...
300683,mail.ru,"Лилия Якупова - Караганда, Карагандинская обла..."
300684,xn----8sbnqchpeeeth.xn--p1ai,Администрация Лесного района Тверской области ...


In [48]:
# 3. Обработка данных

class TextPreprocessing:
    """Класс для обработки текста"""
    @staticmethod
    def language_detect(word: str) -> str:
        russian_letters = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
        return 'ru' if any(letter in russian_letters for letter in word) else 'en'
        
    def remove_non_alphabetic(self, text: str) -> str:
        if not isinstance(text, str):
            return ''
        clear_text = re.sub(r'[^A-Za-zА-Яа-яЁё]+', ' ', text).lower()
        return ' '.join(clear_text.split())
    
    def remove_stop_words(self, text: str, stop_words: list) -> str:
        text = [word for word in text.split() if word not in stop_words]
        return ' '.join(text)
    
    def normalize_text(self, text: str, text_normalizer_ru=MorphAnalyzer(), text_normalizer_en=SnowballStemmer("english")) -> str:
        normalized_text = [text_normalizer_ru.parse(word)[0].normal_form
                           if self.language_detect(word) == 'ru' 
                           else text_normalizer_en.stem(word)
                           for word in text.split()]
        return ' '.join(normalized_text)

In [49]:
stop_words = stopwords.words('russian')

In [51]:
# Обработка title

tp = TextPreprocessing()

df_train['title'] = df_train['title'].apply(lambda text: tp.remove_non_alphabetic(text))
df_train['title'] = df_train['title'].apply(lambda text: tp.remove_stop_words(text, stop_words))
df_train['title'] = df_train['title'].apply(lambda text: tp.normalize_text(text))

df_test['title'] = df_test['title'].apply(lambda text: tp.remove_non_alphabetic(text))
df_test['title'] = df_test['title'].apply(lambda text: tp.remove_stop_words(text, stop_words))
df_test['title'] = df_test['title'].apply(lambda text: tp.normalize_text(text))

,url,title
ID,,
135309,www.kommersant.ru,шестой кассационный суд самара начать работа р...
135310,urexpert.online,индексация алименты случай производиться каков...
135311,imperimeha.ru,женщина империя мех part
135312,national-porn.com,небритый волосатый киска порно страна национал...
135313,2gis.ru,
...,...,...
300682,etp.armtek.ru,armtek запчасть грузовой легковой автомобиль о...
300683,mail.ru,лилия якуп караганда карагандинский область ка...
300684,xn----8sbnqchpeeeth.xn--p1ai,администрация лесной район тверской область го...


In [52]:
stop_words = stopwords.words('english')

In [53]:
# Обработка url

up = TextPreprocessing()

df_train['url'] = df_train['url'].apply(lambda text: tp.remove_non_alphabetic(text))
df_train['url'] = df_train['url'].apply(lambda text: tp.remove_stop_words(text, stop_words))
df_train['url'] = df_train['url'].apply(lambda text: tp.normalize_text(text))

df_test['url'] = df_test['url'].apply(lambda text: tp.remove_non_alphabetic(text))
df_test['url'] = df_test['url'].apply(lambda text: tp.remove_stop_words(text, stop_words))
df_test['url'] = df_test['url'].apply(lambda text: tp.normalize_text(text))

In [55]:
# Объединение
df_train['result'] = df_train['url'] + ' ' + df_train['title']
df_train.drop(['url', 'title'], axis=1, inplace=True)

df_test['result'] = df_test['url'] + ' ' + df_test['title']
df_test.drop(['url', 'title'], axis=1, inplace=True)

In [58]:
# 4. Векторизация текста

tfidf = TfidfVectorizer(ngram_range=(1, 1))
tfidf_train = tfidf.fit_transform(df_train['result'])
tfidf_test = tfidf.transform(df_test['result'])

In [62]:
# 5. Обучение

model = LogisticRegression()
model.fit(tfidf_train, df_train['label'])
df_test['label'] = model.predict(tfidf_test)

In [64]:
# 6. Результат

df_test.drop(['result'], axis=1, inplace=True)

In [65]:
df_test

,label
ID,
135309,0
135310,0
135311,0
135312,1
135313,0
...,...
300682,0
300683,0
300684,0


In [66]:
df_test.to_csv('submission.csv')